In [1]:
import pandas as pd
import requests

In [11]:
#### recuperation donnee statik 

# Envoyer une requête GET à l'API
url = "https://data.nantesmetropole.fr/api/records/1.0/search/?dataset=244400404_stations-velos-libre-service-nantes-metropole-disponibilites&q=&rows=200&facet=banking&facet=bonus&facet=status&facet=contract_name"
response = requests.get(url)

# Vérifier si la requête a réussi
if response.status_code == 200:
    data = response.json()  # Convertir la réponse en JSON
    records = data["records"]  # Obtenir la liste des enregistrements

    # Créer une liste pour stocker les données
    data_list = []

    # Parcourir chaque enregistrement et extraire les champs souhaités
    for record in records:
        fields = record["fields"]
        data_list.append(fields)

    # Créer un DataFrame à partir de la liste de données
    df = pd.DataFrame(data_list)
else:
    print("Échec de la requête")

In [12]:
# recup des donnees manquante sur la deuxieme api 

# Envoyer une requête GET à l'API
url = "https://data.nantesmetropole.fr/api/records/1.0/search/?dataset=244400404_stations-velos-libre-service-nantes-metropole&q=&rows=200&facet=commune&facet=descriptif"
response = requests.get(url)

# Vérifier si la requête a réussi
if response.status_code == 200:
    data = response.json()  # Convertir la réponse en JSON
    records = data["records"]  # Obtenir la liste des enregistrements

    # Créer une liste pour stocker les données
    data_list = []

    # Parcourir chaque enregistrement et extraire les champs souhaités
    for record in records:
        fields = record["fields"]
        data_list.append(fields)

    # Créer un DataFrame à partir de la liste de données
    df2 = pd.DataFrame(data_list)
else:
    print("Échec de la requête")

In [13]:
# preparation des deux df pour le merge final du df_final + merge 

df2['numero_station'] = df2['nom'].str.extract(r'(\d+)')
df['number'] = df['number'].astype(str)
df_final = df.merge(df2[['numero_station', 'ligne_tc']], left_on='number', right_on='numero_station', how='left')

In [14]:
# nettoyage du df_final

df_final = df_final.drop(columns=['bonus','banking','contract_name','numero_station'])

In [15]:
df_final.columns

Index(['available_bike_stands', 'bike_stands', 'number', 'address', 'name',
       'status', 'available_bikes', 'position', 'last_update', 'ligne_tc'],
      dtype='object')

In [17]:
# on definit l'ordre des colonnes
ordre_colonnes = ['number','name','address','bike_stands','available_bike_stands','available_bikes','position','last_update','status','ligne_tc']
# reorganisation des colonnes
df_final = df_final.reindex(columns=ordre_colonnes)


In [18]:
df_final

,number,name,address,bike_stands,available_bike_stands,available_bikes,position,last_update,status,ligne_tc
0,85,085-BEL AIR,"102, rue Paul Bellamy",14,11,3,"[47.2263944309861, -1.56002281967663]",2023-06-19T13:10:04+00:00,OPEN,12 - 23 - C2 - LO
1,104,104-ANTILLES,"20, quai des Antilles",24,22,2,"[47.202547, -1.572325]",2023-06-19T13:10:46+00:00,OPEN,NaN
2,120,120-ROND-POINT DE PARIS,"16, boulevard Jules Verne",30,18,12,"[47.234853, -1.534981]",2023-06-19T13:10:02+00:00,OPEN,C1
3,60,060-GARE DE NANTES NORD,27 boulevard Stalingrad,40,33,7,"[47.217212, -1.543517]",2023-06-19T13:10:58+00:00,OPEN,1
4,124,124-PAS ENCHANTÉS,Boulevard des Pas Enchantés - SAINT-SÉBASTIEN-...,25,8,17,"[47.205239, -1.522726]",2023-06-19T13:09:18+00:00,OPEN,NaN
...,...,...,...,...,...,...,...,...,...,...
123,39,039-QUAI MONCOUSU,"1, quai Moncousu - Pont Haudaudine",24,3,21,"[47.2094065738737, -1.55523314463147]",2023-06-19T13:18:40+00:00,OPEN,26 - LN - LO - LS
124,119,119-ZOLA,Place Emile Zola,25,20,5,"[47.213859, -1.586108]",2023-06-19T13:18:40+00:00,OPEN,10 - 11 - 23
125,14,014-SAINT FÉLIX,"35, Boulevard Amiral Courbet",14,13,1,"[47.230334, -1.556133]",2023-06-19T13:18:05+00:00,OPEN,NaN
126,68,068-LIVET,"65, rue Dufour",15,12,3,"[47.2240678794875, -1.54521243421509]",2023-06-19T13:17:23+00:00,OPEN,C1 - LN


In [19]:
# save du df en csv
df_final.to_csv('df_velo.csv',index=False)